In [61]:
# link used for below task
# https://towardsdatascience.com/writing-advanced-sql-queries-in-pandas-1dc494a17afe

# broadcasting 
# https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computation-on-arrays-broadcasting.html

In [3]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

In [4]:
df = pd.DataFrame({'name': ['Ann', 'Ann', 'Ann', 'Bob', 'Bob'], 
                   'destination': ['Japan', 'Korea', 'Switzerland', 
                                   'USA', 'Switzerland'], 
                   'dep_date': ['2019-02-02', '2019-01-01', 
                                '2020-01-11', '2019-05-05', 
                                '2020-01-11'], 
                   'duration': [7, 21, 14, 10, 14]})

In [5]:
df

,name,destination,dep_date,duration
0,Ann,Japan,2019-02-02,7
1,Ann,Korea,2019-01-01,21
2,Ann,Switzerland,2020-01-11,14
3,Bob,USA,2019-05-05,10
4,Bob,Switzerland,2020-01-11,14


## lead, lag

### SQL

In [6]:
# SELECT name
#        , destination
#        , dep_date
#        , duration
#        , LEAD(dep_date) OVER(ORDER BY dep_date, name) AS lead1
#        , LEAD(dep_date, 2) OVER(ORDER BY dep_date, name) AS lead2
#        , LAG(dep_date) OVER(ORDER BY dep_date, name) AS lag1
#        , LAG(dep_date, 3) OVER(ORDER BY dep_date, name) AS lag3
# FROM df

### pandas

In [11]:
df.sort_values(['dep_date', 'name'], inplace=True)

In [15]:
df.assign(
    lead1 = df['dep_date'].shift(-1),
    lead2 = df['dep_date'].shift(-2),
    lag1 = df['dep_date'].shift(1),
    lag2 = df['dep_date'].shift(2)
)

,name,destination,dep_date,duration,lead1,lead2,lag1,lag2
1,Ann,Korea,2019-01-01,21,2019-02-02,2019-05-05,NaN,NaN
0,Ann,Japan,2019-02-02,7,2019-05-05,2020-01-11,2019-01-01,NaN
3,Bob,USA,2019-05-05,10,2020-01-11,2020-01-11,2019-02-02,2019-01-01
2,Ann,Switzerland,2020-01-11,14,2020-01-11,NaN,2019-05-05,2019-02-02
4,Bob,Switzerland,2020-01-11,14,NaN,NaN,2020-01-11,2019-05-05


## Date/datetime: DATENAME(), DATEDIFF(), DATEADD()

### SQL

In [16]:
# SELECT name
#        , destination
#        , dep_date
#        , duration
#        , DATENAME(WEEKDAY, dep_date) AS day
#        , DATENAME(MONTH, dep_date) AS month
#        , DATEDIFF(DAY,  
#                   LAG(dep_date) OVER(ORDER BY dep_date, name), 
#                   dep_date) AS diff
#        , DATEADD(DAY, day, dep_date) AS arr_date
# FROM df

## pandas

In [44]:
# converting to date-time format

df['dep_date'] = pd.to_datetime(df['dep_date'])

df['duration'] = pd.to_timedelta(df['duration'], 'D')

In [22]:
df

,name,destination,dep_date,duration
1,Ann,Korea,2019-01-01,21 days
0,Ann,Japan,2019-02-02,7 days
3,Bob,USA,2019-05-05,10 days
2,Ann,Switzerland,2020-01-11,14 days
4,Bob,Switzerland,2020-01-11,14 days


In [26]:
df['dep_date'].dt.day # DATENAME(WEEKDAY, dep_date) AS day

1     1
0     2
3     5
2    11
4    11
Name: dep_date, dtype: int64

In [28]:
# df['dep_date'].dt.day_name()
df['dep_date'].dt.month_name()

1     January
0    February
3         May
2     January
4     January
Name: dep_date, dtype: object

In [42]:
df.assign(
    lag1 = df['dep_date'].shift(),                   # lag('duration') over()
    day = df['dep_date'].dt.day_name(),              # DATENAME(WEEKDAY, dep_date) AS day
    month = df['dep_date'].dt.month_name(),          # DATENAME(MONTH, dep_date) AS month
    diff = df['dep_date'] - df['dep_date'].shift(),  # DATEDIFF(DAY, LAG(dep_date) OVER(ORDER BY dep_date, name) # already it is sorted in df
    arr_date = df['dep_date'] + df['duration']       # DATEADD(DAY, day, dep_date) AS arr_date
) #.dtypes # --> to check the date types

,name,destination,dep_date,duration,lag1,day,month,diff,arr_date
1,Ann,Korea,2019-01-01,21 days,NaT,Tuesday,January,NaT,2019-01-22
0,Ann,Japan,2019-02-02,7 days,2019-01-01,Saturday,February,32 days,2019-02-09
3,Bob,USA,2019-05-05,10 days,2019-02-02,Sunday,May,92 days,2019-05-15
2,Ann,Switzerland,2020-01-11,14 days,2019-05-05,Saturday,January,251 days,2020-01-25
4,Bob,Switzerland,2020-01-11,14 days,2020-01-11,Saturday,January,0 days,2020-01-25


## Ranking: ROW_NUMBER(), RANK(), DENSE_RANK()

### SQL

In [43]:
# SELECT name
#        , destination
#        , dep_date
#        , duration
#        , ROW_NUMBER() OVER(ORDER BY duration, name) AS row_number_d
#        , RANK() OVER(ORDER BY duration) AS rank_d
#        , DENSE_RANK() OVER(ORDER BY duration) AS dense_rank_d
# FROM df

## pandas

In [52]:
df.sort_values(['duration', 'name']).assign(
    row_number_d = df['duration'].rank(method='first').astype(int), # ROW_NUMBER() OVER(ORDER BY duration, name) AS row_number_d
    rank_d = df['duration'].rank(method='min').astype(int),         # RANK() OVER(ORDER BY duration) AS rank_d
    dense_rank_d = df['duration'].rank(method='dense').astype(int)  # DENSE_RANK() OVER(ORDER BY duration) AS dense_rank_d
)

,name,destination,dep_date,duration,row_number_d,rank_d,dense_rank_d
0,Ann,Japan,2019-02-02,7 days,1,1,1
3,Bob,USA,2019-05-05,10 days,2,2,2
2,Ann,Switzerland,2020-01-11,14 days,3,3,3
4,Bob,Switzerland,2020-01-11,14 days,4,3,3
1,Ann,Korea,2019-01-01,21 days,5,5,4


## Aggregate window functions & partitioning

### SQL

In [53]:
# SELECT name
#        , destination
#        , dep_date 
#        , duration
#        , MAX(duration) OVER() AS max_dur
#        , SUM(duration) OVER() AS sum_dur
#        , AVG(duration) OVER(PARTITION BY name) AS avg_dur_name
#        , SUM(duration) OVER(PARTITION BY name ORDER BY dep_date
#                             RANGE BETWEEN UNBOUNDED PRECEDING
#                             AND CURRENT ROW) AS cum_sum_dur_name
# FROM df

### pandas

In [60]:
df.assign(
    max_dur = df['duration'].max(),                                  # MAX(duration) OVER() AS max_dur
    sum_dur = df['duration'].sum(),                                  # SUM(duration) OVER() AS sum_dur
    avg_dur_name = df.groupby('name')['duration'].transform('mean'), # AVG(duration) OVER(PARTITION BY name) AS avg_dur_name
    cum_sum_dur_name = df.sort_values('dep_date')
                         .groupby('name')['duration']
                         .transform('cumsum')   # SUM(duration) OVER(PARTITION BY name ORDER BY dep_date
                                                #                      RANGE BETWEEN UNBOUNDED PRECEDING
                                                #                      AND CURRENT ROW) AS cum_sum_dur_name
)

,name,destination,dep_date,duration,max_dur,sum_dur,avg_dur_name,cum_sum_dur_name
1,Ann,Korea,2019-01-01,21 days,21 days,66 days,14 days,21 days
0,Ann,Japan,2019-02-02,7 days,21 days,66 days,14 days,28 days
3,Bob,USA,2019-05-05,10 days,21 days,66 days,12 days,10 days
2,Ann,Switzerland,2020-01-11,14 days,21 days,66 days,14 days,42 days
4,Bob,Switzerland,2020-01-11,14 days,21 days,66 days,12 days,24 days


In [67]:
df.sort_values(['name', 'dep_date']).assign(
    max_dur = df['duration'].max(),
    sum_dur = df['duration'].sum(),
    avg_dur_name = df.groupby('name')['duration'].transform('mean'),
    sum_sum_dur_name = df.groupby('name')['duration'].transform('cumsum')
)

,name,destination,dep_date,duration,max_dur,sum_dur,avg_dur_name,sum_sum_dur_name
1,Ann,Korea,2019-01-01,21 days,21 days,66 days,14 days,21 days
0,Ann,Japan,2019-02-02,7 days,21 days,66 days,14 days,28 days
2,Ann,Switzerland,2020-01-11,14 days,21 days,66 days,14 days,42 days
3,Bob,USA,2019-05-05,10 days,21 days,66 days,12 days,10 days
4,Bob,Switzerland,2020-01-11,14 days,21 days,66 days,12 days,24 days


In [65]:
df.groupby('name')['duration'].mean()

name
Ann   14 days
Bob   12 days
Name: duration, dtype: timedelta64[ns]

## final

### SQL

In [68]:
# SELECT ROW_NUMBER() OVER(PARTITION BY name 
#                          ORDER BY dep_date) AS number
#        , name
#        , destination
#        , DATENAME(MONTH, dep_date) AS dep_month
#        , dep_date
#        , DATEADD(DAY, duration, dep_date) AS arr_date
#        , LEAD(dep_date) OVER(PARTITION BY NAME 
#                              ORDER BY dep_date) AS next_dep_date
#        , DATEDIFF(DAY, 
#                   dep_date, 
#                   LEAD(dep_date) OVER(PARTITION BY NAME 
#                                       ORDER BY dep_date)) AS gap
#        , duration
#        , AVG(1.0 * duration) OVER() AS avg_dur
#        , AVG(1.0 * duration) OVER(PARTITION BY name) AS avg_dur_name
#        , SUM(duration) OVER(PARTITION BY name ORDER BY dep_date
#                             RANGE BETWEEN UNBOUNDED PRECEDING 
#                             AND CURRENT ROW) AS cum_sum_dur_name   
# FROM df
# ORDER BY name, dep_date

### pandas

In [73]:
# converting to date-time format

df['dep_date'] = pd.to_datetime(df['dep_date'])
df['duration'] = pd.to_timedelta(df['duration'], 'D')

df.sort_values(['name', 'dep_date'], inplace=True)

In [93]:
df.assign(
    number = df.groupby('name')['dep_date'].rank(method='first').astype(int),
    dep_month = df['dep_date'].dt.month_name(),
    arr_date = df['dep_date'] + df['duration'],
    next_dep_date = df.groupby('name')['dep_date'].transform(lambda x: x.shift(-1)),
    gap = df.groupby('name')['dep_date'].transform(lambda x: (x.shift(-1) - df['dep_date'])),
    avg_dur = df['duration'].mean(),
    avg_dur_name = df.groupby('name')['duration'].mean(), # --> this will not work
    avg_dur_name_temp = df.groupby('name')['duration'].transform('mean'),
    cum_sum_dur_name = df.groupby('name')['duration'].transform('cumsum'), # or
    cum_sum_dur_name_temp = df.groupby('name')['duration'].apply(lambda x: x.cumsum())
)

,name,destination,dep_date,duration,number,dep_month,arr_date,next_dep_date,gap,avg_dur,avg_dur_name,avg_dur_name_temp,cum_sum_dur_name,cum_sum_dur_name_temp
1,Ann,Korea,2019-01-01,21 days,1,January,2019-01-22,2019-02-02,32 days,13 days 04:48:00,NaT,14 days,21 days,21 days
0,Ann,Japan,2019-02-02,7 days,2,February,2019-02-09,2020-01-11,343 days,13 days 04:48:00,NaT,14 days,28 days,28 days
2,Ann,Switzerland,2020-01-11,14 days,3,January,2020-01-25,NaT,NaT,13 days 04:48:00,NaT,14 days,42 days,42 days
3,Bob,USA,2019-05-05,10 days,1,May,2019-05-15,2020-01-11,251 days,13 days 04:48:00,NaT,12 days,10 days,10 days
4,Bob,Switzerland,2020-01-11,14 days,2,January,2020-01-25,NaT,NaT,13 days 04:48:00,NaT,12 days,24 days,24 days
